<a href="https://colab.research.google.com/github/yoonjh25/HTML-CSS/blob/main/deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
import json

# Google 드라이브에 있는 이미지 폴더 경로 및 JSON 파일 경로 설정
base_image_folder = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신'
json_folder = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/[라벨]실신_1'

# YOLO 형식으로 변환된 라벨 파일을 저장할 폴더
yolo_label_folder = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels'
os.makedirs(yolo_label_folder, exist_ok=True)

# 클래스 매핑: 사람만 고려 (아이도 사람이므로 모두 'person'으로 처리)
class_mapping = {
    "person": 0  # 'person' 카테고리만 남김
}

# JSON 파일을 읽어 YOLO 형식의 라벨 파일을 생성하는 함수
def convert_json_to_yolo(json_file_path, image_folder, image_subfolder):
    # JSON 파일 읽기
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # 이미지 메타데이터 가져오기
    image_width = data['metadata']['width']
    image_height = data['metadata']['height']

    # 각 프레임에 대한 YOLO 라벨 파일 생성
    for frame in data['frames']:
        image_file_name = frame['image']  # 예: frame_1.jpg
        yolo_label_file = os.path.join(yolo_label_folder, image_subfolder + '_' + image_file_name.replace('.jpg', '.txt'))

        with open(yolo_label_file, 'w') as label_file:
            for annotation in frame['annotations']:
                category = annotation['category']['code']
                # 'person'만 처리하고 나머지는 무시
                if category not in class_mapping:
                    continue

                # 클래스 ID 가져오기 (모두 'person'으로 매핑됨)
                class_id = class_mapping[category]

                # Bounding Box 좌표 및 크기 정보
                x = annotation['label']['x']
                y = annotation['label']['y']
                width = annotation['label']['width']
                height = annotation['label']['height']

                # YOLO 형식으로 변환
                x_center = (x + width / 2) / image_width
                y_center = (y + height / 2) / image_height
                box_width = width / image_width
                box_height = height / image_height

                # YOLO 라벨 형식으로 저장
                label_file.write(f"{class_id} {x_center} {y_center} {box_width} {box_height}\n")

# 이미지 폴더 내 모든 하위 폴더에 대해 처리
for subfolder in os.listdir(base_image_folder):
    subfolder_path = os.path.join(base_image_folder, subfolder)

    if os.path.isdir(subfolder_path):
        # JSON 파일 경로 (하위 폴더 이름에 맞는 JSON 파일을 찾음)
        json_file_name = f"annotation_{subfolder}.json"
        json_file_path = os.path.join(json_folder, json_file_name)

        if os.path.exists(json_file_path):
            # 해당 폴더 내 이미지 파일을 YOLO 형식으로 변환
            convert_json_to_yolo(json_file_path, subfolder_path, subfolder)
        else:
            print(f"JSON 파일을 찾을 수 없습니다: {json_file_path}")


JSON 파일을 찾을 수 없습니다: /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/[라벨]실신_1/annotation_[라벨]실신_1.json


In [19]:
import os

# 이미지와 라벨 파일들이 있는 경로 설정
image_folder = '/content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신'
label_folder = '/content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels'

# 이미지 파일에 대응하는 라벨 파일 이름을 변경하는 함수
def rename_label_files(image_folder, label_folder):
    for root, dirs, files in os.walk(image_folder):
        for file in files:
            if file.endswith('.jpg'):
                # 이미지 파일 이름 가져오기 (예: frame_1.jpg)
                image_file_name = file

                # 이미지 파일에 대응하는 라벨 파일 (예: 12341_frame_1.txt)
                label_file_name = os.path.basename(root) + '_' + image_file_name.replace('.jpg', '.txt')

                # 새로 변경할 라벨 파일 이름 (예: frame_1.txt)
                new_label_file_name = image_file_name.replace('.jpg', '.txt')

                # 라벨 파일 경로
                old_label_file_path = os.path.join(label_folder, label_file_name)
                new_label_file_path = os.path.join(label_folder, new_label_file_name)

                # 라벨 파일 이름 변경
                if os.path.exists(old_label_file_path):
                    os.rename(old_label_file_path, new_label_file_path)
                    print(f'Renamed: {old_label_file_path} -> {new_label_file_path}')
                else:
                    print(f'Label file not found: {old_label_file_path}')

# 라벨 파일 이름 변경 실행
rename_label_files(image_folder, label_folder)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Renamed: /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/2660501_frame_5066.txt -> /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5066.txt
Renamed: /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/2660501_frame_5076.txt -> /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5076.txt
Renamed: /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/2660501_frame_5086.txt -> /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5086.txt
Renamed: /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/2660501_frame_5096.txt -> /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5096.txt
Renamed: /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/2660501_frame_5106.txt -> /content/drive/MyDrive/deep/지하철 역사 내

In [24]:
import os

label_folder = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels'

# yolo_labels 폴더의 모든 파일 출력
print("yolo_labels 폴더의 파일 목록:")
for label_file in os.listdir(label_folder):
    print(label_file)

# 특정 라벨 파일 확인
label_to_check = 'frame_5770.txt'
if os.path.exists(os.path.join(label_folder, label_to_check)):
    print(f"{label_to_check} 파일이 존재합니다.")
else:
    print(f"{label_to_check} 파일을 찾을 수 없습니다.")


yolo_labels 폴더의 파일 목록:
frame_5770.txt 파일을 찾을 수 없습니다.


In [23]:
import os
import shutil

# YOLO 라벨 파일들이 있는 폴더
label_folder = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels'

# 이미지 폴더 경로
image_root = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신'

# 모든 이미지 폴더 순회
for folder in os.listdir(image_root):
    folder_path = os.path.join(image_root, folder)
    # 이미지 폴더라면(폴더 안에 frame_*.jpg가 있는 폴더라면)
    if os.path.isdir(folder_path) and folder != 'yolo_labels':  # 'yolo_labels' 폴더 제외
        # 해당 폴더 안의 모든 이미지 파일 순회
        for image_file in os.listdir(folder_path):
            if image_file.endswith('.jpg'):
                # 이미지 파일에 대응하는 라벨 파일 이름 생성
                label_file = image_file.replace('.jpg', '.txt')
                label_src = os.path.join(label_folder, label_file)
                label_dst = os.path.join(folder_path, label_file)

                # 라벨 파일이 존재하면 해당 폴더로 이동
                if os.path.exists(label_src):
                    shutil.move(label_src, label_dst)
                    print(f"{label_file} 라벨 파일을 {folder_path}로 이동시켰습니다.")
                else:
                    print(f"라벨 파일 {label_src}를 찾을 수 없습니다.")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5066.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5076.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5086.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5096.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5106.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5116.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5126.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo_labels/frame_5136.txt를 찾을 수 없습니다.
라벨 파일 /content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/yolo

In [6]:
!git clone https://github.com/ultralytics/yolov5  # YOLOv5 저장소 클론
%cd yolov5
!pip install -r requirements.txt  # 필요한 라이브러리 설치


Cloning into 'yolov5'...
remote: Enumerating objects: 16995, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 16995 (delta 101), reused 115 (delta 54), pack-reused 16805 (from 1)
Receiving objects: 100% (16995/16995), 15.72 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (11639/11639), done.
/content/yolov5/yolov5


In [20]:
# dataset.yaml 파일 생성
yaml_content = """
train: /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/2131424 # 학습용 이미지 경로
val: /content/drive/MyDrive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신/2660264  # 검증용 이미지 경로

nc: 1  # 클래스 수 (실신 관련 행동만 감지하므로 1)
names: ['person']  # 클래스 이름 (사람만 해당)
"""

# dataset.yaml 파일로 저장
with open('/content/drive/MyDrive/deep/dataset.yaml', 'w') as f:
    f.write(yaml_content)

print("dataset.yaml 파일이 저장되었습니다.")


dataset.yaml 파일이 저장되었습니다.


In [21]:
!python train.py --img 640 --batch 16 --epochs 100 --data /content/drive/MyDrive/deep/dataset.yaml --weights yolov5s.pt


2024-10-17 12:40:22.922848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 12:40:22.945017: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 12:40:22.956443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/deep/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scal

In [9]:
!python val.py --weights runs/train/exp/weights/best.pt --data /content/yolov5/dataset.yaml --img 640


Traceback (most recent call last):
  File "/content/yolov5/yolov5/val.py", line 603, in <module>
    opt = parse_opt()
  File "/content/yolov5/yolov5/val.py", line 541, in parse_opt
    opt.data = check_yaml(opt.data)  # check YAML
  File "/content/yolov5/yolov5/utils/general.py", line 479, in check_yaml
    return check_file(file, suffix)
  File "/content/yolov5/yolov5/utils/general.py", line 507, in check_file
    assert len(files), f"File not found: {file}"  # assert file was found
AssertionError: File not found: /content/yolov5/dataset.yaml


In [5]:
import zipfile
import os

file_path = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/[원천]실신_1.zip'

# 압축을 해제할 폴더 경로
output_dir = '/content/drive/My Drive/deep/지하철 역사 내 CCTV 이상행동 영상/Training/실신/실신'

# 폴더가 없으면 새로 만듭니다
os.makedirs(output_dir, exist_ok=True)

# zip 파일 열고 압축 해제
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print('압축 해제 완료!')


압축 해제 완료!


YOLO 사용


In [ ]:
import os
import cv2
from tqdm import tqdm

# 파일 경로 설정
root_dir = "/content/drive/My Drive/deep/unzipped"  # Google 드라이브에 마운트된 경로
save_image_root = "/content/drive/My Drive/deep/extracted_frames"  # 프레임을 저장할 경로
os.makedirs(save_image_root, exist_ok=True)

# 프레임 추출을 위한 함수
def extract_frames_from_video(video_path, save_dir, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_name = os.path.basename(video_path)
    count = 0  # 저장할 이미지 파일의 번호

    # 프레임 추출
    for frame_number in tqdm(range(frame_count), desc=f"Processing {video_name}"):
        ret, image = cap.read()
        if not ret:
            break  # 더 이상 읽을 수 있는 프레임이 없으면 종료

        # 30프레임마다 이미지 추출
        if frame_number % frame_interval == 0:
            save_path = os.path.join(save_dir, f"{video_name}_frame{count:06d}.jpg")
            cv2.imwrite(save_path, image)
            count += 1

    cap.release()

# 하위 디렉토리를 재귀적으로 탐색하여 비디오 파일을 처리하는 함수
def process_all_videos_recursive(root_dir, save_root, frame_interval=30):
    for root, dirs, files in os.walk(root_dir):  # 재귀적으로 모든 하위 폴더 탐색
        for file in files:
            if file.endswith('.mp4'):  # 비디오 파일만 처리
                video_path = os.path.join(root, file)
                relative_folder = os.path.relpath(root, root_dir)  # 저장할 폴더 구조 유지
                save_dir = os.path.join(save_root, relative_folder)
                os.makedirs(save_dir, exist_ok=True)  # 저장할 디렉터리 생성
                extract_frames_from_video(video_path, save_dir, frame_interval)

# 실행
process_all_videos_recursive(root_dir, save_image_root)


Processing 100-4_cam02_swoon01_place02_day_spring.mp4: 100%|██████████| 9216/9216 [05:15<00:00, 29.18it/s]
Processing 100-4_cam01_swoon01_place02_day_spring.mp4: 100%|██████████| 9302/9302 [05:21<00:00, 28.89it/s]
Processing 100-4_cam02_swoon01_place02_day_summer.mp4: 100%|██████████| 9151/9151 [05:14<00:00, 29.14it/s]
Processing 100-4_cam01_swoon01_place02_day_summer.mp4: 100%|██████████| 9337/9337 [05:21<00:00, 29.07it/s]
Processing 103-5_cam01_swoon01_place04_day_summer.mp4:  92%|█████████▏| 8970/9789 [06:47<00:37, 22.03it/s]


KeyboardInterrupt: 